# FLARE - LAMMPS tutorial

Yu Xie (xiey@g.harvard.edu)

## What will you learn in this tutorial

In the [FLARE tutorial](https://colab.research.google.com/drive/1rZ-p3kN5CJbPJgD8HuQHSc7ecmwZYse6), we learned how to run on-the-fly training with FLARE and MD engine from ASE. In this tutorial, you will learn how to

- Deploy the trained FLARE Bayesian force field to large-scale MD in LAMMPS 👉 [run LAMMPS MD with FLARE pair style and uncertainty](#scrollTo=Run_MD_in_LAMMPS_with_uncertainty)

- Use a faster and more flexible MD for the on-the-fly training 👉 [train FLARE force field on-the-fly with LAMMPS MD](#scrollTo=Bayesian_active_learning_with_FLARE_and_LAMMPS_MD)

> Why do we use LAMMPS MD for on-the-fly training?
>
> 1. ASE only provides very limited MD engines (Verlet, Langevin, NPT, etc.) and functionalities.
>
> 2. The energy/forces/stress are predicted by sparse GP force field, whose computational cost grows with increasing training set size, and can be slow as the training proceeds.





# Installation

## Install FLARE

The installation will take a few minutes.

In [ ]:
# Install a working copy of lapack/lapacke.
! pip uninstall -y mkl mkl-devel mkl-include
! sudo apt remove *mkl*
! sudo apt install liblapacke liblapacke-dev

# Switch the Colab C++ compiler to g++-9.
! sudo add-apt-repository -y ppa:ubuntu-toolchain-r/test
! sudo apt update
! sudo apt install gcc-9 g++-9
! update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 50
! update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 50

!git clone -b 1.3.3 https://github.com/mir-group/flare.git
!cd flare && pip install .

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libmkl-intel-thread' for glob '*mkl*'
Note, selecting 'libmkl-gf-lp64' for glob '*mkl*'
Note, selecting 'libmkl-blacs-intelmpi-lp64' for glob '*mkl*'
Note, selecting 'intel-mkl-linktool' for glob '*mkl*'
Note, selecting 'libmkl-scalapack-lp64' for glob '*mkl*'
Note, selecting 'libmkl-gf-ilp64' for glob '*mkl*'
Note, selecting 'libmkl-gnu-thread' for glob '*mkl*'
Note, selecting 'libmkl-avx2' for glob '*mkl*'
Note, selecting 'libmkl-full-dev' for glob '*mkl*'
Note, selecting 'libmkl-vml-mc' for glob '*mkl*'
Note, selecting 'intel-mkl-cluster' for glob '*mkl*'
Note, selecting 'libmkl-mc' for glob '*mkl*'
Note, selecting 'libmkl-rt' for glob '*mkl*'
Note, selecting 'libmkldnn-dev' for glob '*mkl*'
Note, selecting 'php-srmklive-flysystem-dropbox-v2' for glob '*mkl*'
Note, selecting 'libmkl-pgi-thread' for glob '*mkl*'
Note, selecting 'libmkl-cluster-dev' for glob '*mkl*'
Note, 

## Install LAMMPS with FLARE pair styles and compute commands

The LAMMPS code files for FLARE are in the `flare/lammps_plugins` folder. We go to the folder and run the `install.sh`. Then we go to the `lammps` folder, and run `cmake` and `make` to compile the LAMMPS executable.

This will take a few minutes.

In [ ]:
# compile lammps
# Switch the Colab C++ compiler to g++-9.
# !apt update
# !apt install -y cmake

#!wget "https://github.com/lammps/lammps/archive/stable.zip"
!wget "https://github.com/lammps/lammps/archive/refs/tags/patch_17Feb2022.zip"
!unzip -q patch_17Feb2022.zip
!rm patch_17Feb2022.zip
!mv lammps-patch_17Feb2022 lammps
!cd flare/lammps_plugins && ./install.sh ../../lammps
!cd lammps && mkdir -p build && cd build && cmake ../cmake -DPKG_MACHDYN=yes -DDOWNLOAD_EIGEN3=yes -DPKG_MANYBODY=yes && make -j4

--2023-08-23 09:52:09--  https://github.com/lammps/lammps/archive/refs/tags/patch_17Feb2022.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lammps/lammps/zip/refs/tags/patch_17Feb2022 [following]
--2023-08-23 09:52:09--  https://codeload.github.com/lammps/lammps/zip/refs/tags/patch_17Feb2022
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘patch_17Feb2022.zip’

patch_17Feb2022.zip     [        <=>         ] 124.90M  6.63MB/s    in 20s     

2023-08-23 09:52:29 (6.34 MB/s) - ‘patch_17Feb2022.zip’ saved [130962706]

mv: cannot move 'lammps-patch_17Feb2022' to 'lammps/lammps-patch_17Feb2022': Directory not empty
ln: failed to

# Bayesian active learning with FLARE and LAMMPS MD

## Step 1: Make the initial structure.

We'll simulate an adatom on an aluminum slab to illustrate what happens when one local environment doesn't resemble any of the others in the structure.

We need a file that stores the initial atomic structure for the on-the-fly training.

In [ ]:
# Import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc('font', size=12)
matplotlib.rcParams['figure.dpi'] = 100

# ASE imports
from ase import Atoms, units
from ase.build import supercells
from ase.visualize import view
from ase.build import fcc111, add_adsorbate
from ase.io import read, write

# Create a slab with an adatom.
atoms = fcc111("Al", (4, 4, 6), vacuum=10.0)
add_adsorbate(atoms, "Al", 2.5, "ontop")
n_atoms = len(atoms)

write("Al.xyz", atoms)

view(atoms, viewer='x3d')

ModuleNotFoundError: ignored

## Step 2: Prepare a yaml configuration file

The yaml config file is the input to FLARE to run on-the-fly/offline training. It includes four sections:
1. `supercell`: the file of the initial structure is given. Any format supported by ASE IO can be used.
2. `flare_calc`: set up the sparse GP parameters
3. `dft_calc`: set up the DFT calculator. Any calculator supported by ASE can be used.
4. `otf`: set up the on-the-fly training parameters, including MD.

In [ ]:
! git clone https://github.com/mir-group/FLARE-Tutorials.git

In [ ]:
!cp lammps/potentials/Al_zhou.eam.alloy ./
!cp FLARE-Tutorials/OTF/*.yaml .
!cat lammps.yaml

The flexibility of LAMMPS allows us to use different `fix` commands, and grouping atoms using `region` and `group`. Here is an example
```yaml
    md_kwargs:
        command: "lammps/build/lmp"
        specorder: [Al]
        dump_period: 10
        pair_style: flare
        region:
            - "bottom block INF INF INF INF INF 4.0\n"
        group:
            - "frozen region bottom"
            - "mobile subtract all frozen"
        fix:
            - "1 mobile nvt temp 2500 2500 $(100.0*dt)"
            - "2 frozen setforce 0 0 0"
```

## Step 3: Run on-the-fly training

If you want to use OpenMP parallelization to speed up FLARE, then set
```
export OMP_NUM_THREADS=<number_of_cpus_on_a_single_node>
```

To launch the training, in the command line, run `flare-otf <name>.yaml`.

In [ ]:
!flare-otf lammps.yaml

## Step 4: Output files
In the same folder as you run `flare-otf`, there will be multiple output files dumped during the training.

| File name | Description |
| --------- | ----------- |
|`<output_name>.out` | log file of the OTF training |
|`<output_name>_dft.xyz` | all the DFT computed frames and the corresponding DFT energy/forces/stress |
|  `<output_name>_md.xyz` | complete MD trajectory from the on-the-fly training |
| `<output_name>_dft.pickle`| ASE DFT calculator (used for restarting OTF) |
| `<output_name>_flare.json`| FLARE calculator with training data collected from OTF (used for restarting) |
| `<output_name>_thermo.txt`| thermal outputs from LAMMPS of the complete MD trajectory |
| `<output_name>_atoms.json`| atomic structure at the current step (ASE Atoms object) |
| `<output_name>_checkpt.json`| checkpoint file that saved the OTF information at the current step, and can be used to restart an OTF training |
| `lmp.flare`| LAMMPS potential file generated by FLARE, and used for MD |
| `L_inv_lmp.flare`, `sparse_desc_lmp.flare`| coefficient files used in LAMMPS for uncertainty predictions |
| `<output_name>_ckpt_<n>` | (optional) If you set `write_model: 4`,  those folders back up the checkpoint files at step n |

## Step 5: Postprocessing

In `myotf_thermo.txt`, the columns are
step, temp, ke, pe, etotal, pxx, pyy, pzz, pyz, pxz, pxy, c_MaxUnc

In [ ]:
otf_data = np.loadtxt("myotf_thermo.txt")
steps = otf_data[:, 0] # 1 fs/step
max_uncertainty = otf_data[:, -1]
threshold = 0.02

# Plot maximal atomic uncertainty at each MD step
plt.plot(steps, max_uncertainty, color="blue")

# Plot the uncertainty threshold of calling DFT
plt.plot(steps, threshold * np.ones_like(steps), "--", color="red", label="Threshold")

# Plot DFT calls with grey dash lines
steps_above_threshold = steps[max_uncertainty > threshold]
for dft_step in steps_above_threshold:
    if dft_step == steps_above_threshold[0]:
        label = "DFT"
    else:
        label = None
    plt.axvline(dft_step, linestyle=":", color="grey", label=label)

plt.yscale("log")
plt.xlabel("Simulation time (fs)")
plt.ylabel("Maximal uncertainty")
plt.legend()

In [ ]:
temperatures = otf_data[:, 1]
plt.plot(steps, temperatures)

In [ ]:
potential_energies = otf_data[:, 3]
plt.plot(steps, potential_energies)

# Run MD in LAMMPS with uncertainty

## Coefficient files for LAMMPS

After the training is done, you will probably want to deploy the force field to large-scale molecular dynamics in LAMMPS. You need the coefficient files for FLARE pair styles (and compute commands if the uncertainty is needed to be calculated).

The following files are needed for running FLARE in LAMMPS:

- **Force field**: coefficient file `lmp.flare` for pair_flare energy/forces/stress. Can be a different file name. Used as

  ```
  pair_style	flare
  pair_coeff	* * lmp.flare
  ```

- **(Optional) SGP uncertainty**: coefficient files `sparse_desc_lmp.flare` and `L_inv_lmp.flare` for sparse GP uncertainty calculation. Can be different file names. Used as

  ```
  compute unc all flare/std/atom L_inv_lmp.flare sparse_desc_lmp.flare
  ```

  Note: this is the exact SGP uncertainty, and its computational cost grows quadratically with the sparse set size.

- **(Optional) Mapped uncertainty**: coefficient file  `mapped_unc_lmp.flare` for mapped uncertainty calculation. Can be a different file name. Used as

  ```
  compute unc all flare/std/atom map_unc_lmp.flare
  ```

  Note:
  1. This is the mapped uncertainty which is an approximation of the exact SGP uncertainty with a much cheaper computational cost comparable to the force field. See details in [this paper](https://arxiv.org/abs/2203.03824).
  2. Basically, when there are two coefficient files after `compute unc all flare/std/atom` it will calculate the exact SGP uncertainty, when there is one coefficient file it will calculate the mapped uncertainty.

## Generate coefficient files

- If you have set `use_mapping: True` in the on-the-fly/offline training, then at the end of the training you will have `lmp.flare`, `sparse_desc_lmp.flare` and `L_inv_lmp.flare` available in the current directory.

- If you do not have the coefficient files, you will need the json file of the sparse GP calculator which can be found in the same folder as the training. In the above example, it is `Al_flare.json`. Then run the simple script below. You can put your name as the contributor, and then `lmp.flare`, `sparse_desc_lmp.flare` and `L_inv_lmp.flare` will be generated in the current directory.

> **NOTE**: This script is useful for constructing potential files. You should keep it and can use it after restarting an active learning trajectory, offline training, etc.


In [ ]:
from flare.bffs.sgp.calculator import SGP_Calculator

sgp_calc, _ = SGP_Calculator.from_file("myotf_flare.json")
sgp_calc.build_map("lmp.flare", "your_name", map_uncertainty=False)

If `map_uncertainty=True`, the mapped uncertainty coefficient file `map_unc_lmp.flare` will be generated instead of `sparse_desc_lmp.flare` and `L_inv_lmp.flare`. It might take a while because a new SGP with `power=1` kernel will be constructed and mapped.

## Run MD with uncertainty

Prepare a LAMMPS data file


> **Note**: if you are using ASE to write a LAMMPS data file, be careful of the species order. Make sure you set up the `specorder` to the list of the elements with the same order specified in the yaml file of the on-the-fly training. Otherwise, ASE will assign type ID to atoms based on the order of their chemical numbers by default. The inconsistency between the atomic type ID and the potential type order will make the energy/forces/stress prediction wrong.



In [ ]:
! mkdir lammps_run

from ase.io import read, write
atoms = read("Al.xyz")
write("lammps_run/Al.data", atoms, format="lammps-data", specorder=["Al"])

Copy LAMMPS coefficient files for FLARE potential and uncertainty to the directory of MD run

In [ ]:
!cp lmp.flare L_inv_lmp.flare sparse_desc_lmp.flare lammps_run/

Prepare a LAMMPS input file. Here we set up an NVT run at 300K.

In [ ]:
!cp FLARE-Tutorials/OTF/lammps.in lammps_run/
!cat lammps_run/lammps.in

Run LAMMPS

In [ ]:
!cd lammps_run && ../lammps/build/lmp -in lammps.in

Using Python to call LAMMPS

## Visualize LAMMPS MD trajectory

### Color Atoms by Uncertainty

If you just run the on-the-fly training with PyLAMMPS or you used this
```
compute unc all flare/std/atom L_inv_lmp.flare sparse_desc_lmp.flare
compute MaxUnc all reduce max c_unc

dump dump_all all custom 10 output.dump id type x y z fx fy fz c_unc
thermo_style custom step temp pe etotal press vol c_MaxUnc
```
in your LAMMPS MD input script, then the uncertainty information is dumped. In the dumped trajectory, the atomic uncertainty is recorded in the last column `c_unc`. In the thermostat file (LAMMPS `log` file), the maximal atomic uncertainty in each step is recorded in the last column `c_MaxUnc`.

You can visualize the dumped trajectory `output.dump` in softwares like [OVITO](https://www.ovito.org), where the atomic uncertainty `c_unc` can be used for `color coding`.

<img src="https://github.com/mir-group/FLARE-Tutorials/raw/master/APS-2020/al.gif" width="100%">

### Plot Thermostat

Here we plot the temperature and maximal atomic uncertainty with the simulation time. The data is extracted from the LAMMPS log file, using a python package `lammps-logfile`

In [ ]:
!pip install lammps-logfile

import lammps_logfile

log = lammps_logfile.File("lammps_run/log.lammps")
plt.plot(log.get("Step"), log.get("Temp"))
plt.xlabel("Step")
plt.ylabel("Temperature (K)")

The maximal atomic uncertainty can be used to monitor how confident the model is, and how far the current step gets away from the training data. It is very helpful for monitoring if unphysical behavior happens.

In [ ]:
plt.plot(log.get("Step"), log.get("c_MaxUnc"))
plt.xlabel("Step")
plt.ylabel("Max uncertainty")